In [1]:
#import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pathlib import Path
from EHydro_TreeUnet.trainers import TreeProjectorTrainer
from torchsparse.nn import functional as F

F.set_kmap_mode("hashmap_on_the_fly")

ImportError: cannot import name 'TensorOpParams' from 'cumm.gemm.algospec.core' (/home/samuel/.local/lib/python3.10/site-packages/cumm/gemm/algospec/core.py)

In [ ]:
TRAINING = True

DATASET_FOLDER = Path.home() / 'datasets/MixedDataset'
WEIGHTS_FILE = 'tree_projector_instance_VS-0.2_DA-48_E-3_v2.pth'
CHECKPOINT_FILE = None
FEAT_KEYS = ['intensity']
IGNORE_CLASS = []
CHANNELS = [16, 32, 64, 128]
LATENT_DIM = 256
INSTANCE_DENSITY = 0.01
CENTROID_THRES = 0.1
PEAK_RADIUS = 2
MIN_SCORE_FOR_CENTROID = 0.0
DESCRIPTOR_DIM = 32
TRAIN_PCT = 0.9
VOXEL_SIZE = 0.2
DATA_AUGMENTATION_COEF = 48
EPOCHS = 3
SEMANTIC_LOSS_COEF = 1.0
CENTROID_LOSS_COEF = 1.0
INSTANCE_LOSS_COEF = 1.0
BATCH_SIZE = 1

In [ ]:
def smooth(arr: np.ndarray, window: int) -> np.ndarray:
    if window <= 1:
        return arr

    kernel = np.ones(window, dtype=float)

    if arr.ndim == 1:
        denom = np.convolve(np.ones_like(arr), kernel, mode="same")
        return np.convolve(arr, kernel, mode="same") / denom

    # 2-D: suavizar cada columna por separado
    smoothed = np.empty_like(arr, dtype=float)
    denom = np.convolve(np.ones(arr.shape[0]), kernel, mode="same")
    for c in range(arr.shape[1]):
        smoothed[:, c] = np.convolve(arr[:, c], kernel, mode="same") / denom
    return smoothed
        
def gen_charts(trainer, losses, stats, training: bool, window: int = 10, ignore_class = []):
    keys = stats[0].keys()
    stats = {k: np.array([d[k] for d in stats]) for k in keys}

    losses_s      = np.clip(smooth(np.asarray(losses), window), 0.0, 10.0)
    miou_s        = smooth(stats['miou'], window)
    iou_s         = smooth(np.asarray(stats['iou_per_class']), window)
    prec_macro_s  = smooth(stats['precision_macro'], window)
    prec_s        = smooth(np.asarray(stats['precision_per_class']), window)
    recall_macro_s= smooth(stats['recall_macro'], window)
    recall_s      = smooth(np.asarray(stats['recall_per_class']), window)
    f1_macro_s    = smooth(stats['f1_macro'], window)
    f1_s          = smooth(np.asarray(stats['f1_per_class']), window)

    # --- 1. Loss -------------------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(losses_s, label=f"{'Training' if training else 'Inference'} Loss (MA{window})")
    plt.xlabel("Step"); plt.ylabel("Loss")
    plt.title(f"Loss evolution during {'Training' if training else 'Inference'}")
    plt.legend(); plt.grid(True); plt.show()

    # --- 2. mIoU (macro) -----------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(miou_s, label=f"{'Training' if training else 'Inference'} mIoU (MA{window})")
    plt.xlabel("Step"); plt.ylabel("mIoU")
    plt.title(f"mIoU evolution during {'Training' if training else 'Inference'}")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- 3. IoU per class ----------------------------------------------------
    plt.figure(figsize=(10, 5))
    for c in range(trainer.dataset.num_classes):
        if trainer.dataset.class_names[c] in ignore_class:
            continue
        
        plt.plot(iou_s[:, c], label=trainer.dataset.class_names[c])
    plt.xlabel("Step"); plt.ylabel("IoU")
    plt.title(f"IoU evolution during {'Training' if training else 'Inference'} (MA{window})")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- 4. Precision (macro) -----------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(prec_macro_s, label=f"{'Training' if training else 'Inference'} precision (MA{window})")
    plt.xlabel("Step"); plt.ylabel("Precision")
    plt.title(f"Precision evolution during {'Training' if training else 'Inference'}")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- 5. Precision per class ---------------------------------------------
    plt.figure(figsize=(10, 5))
    for c in range(trainer.dataset.num_classes):
        if trainer.dataset.class_names[c] in ignore_class:
            continue
        
        plt.plot(prec_s[:, c], label=trainer.dataset.class_names[c])
    plt.xlabel("Step"); plt.ylabel("Precision")
    plt.title(f"Precision evolution during {'Training' if training else 'Inference'} (MA{window})")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- 6. Recall (macro) ---------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(recall_macro_s, label=f"{'Training' if training else 'Inference'} recall (MA{window})")
    plt.xlabel("Step"); plt.ylabel("Recall")
    plt.title(f"Recall evolution during {'Training' if training else 'Inference'}")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- 7. Recall per class -------------------------------------------------
    plt.figure(figsize=(10, 5))
    for c in range(trainer.dataset.num_classes):
        if trainer.dataset.class_names[c] in ignore_class:
            continue
        
        plt.plot(recall_s[:, c], label=trainer.dataset.class_names[c])
    plt.xlabel("Step"); plt.ylabel("Recall")
    plt.title(f"Recall evolution during {'Training' if training else 'Inference'} (MA{window})")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- 8. F1 (macro) -------------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(f1_macro_s, label=f"{'Training' if training else 'Inference'} F1 (MA{window})")
    plt.xlabel("Step"); plt.ylabel("F1")
    plt.title(f"F1 evolution during {'Training' if training else 'Inference'}")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- 9. F1 per class -----------------------------------------------------
    plt.figure(figsize=(10, 5))
    for c in range(trainer.dataset.num_classes):
        if trainer.dataset.class_names[c] in ignore_class:
            continue
        
        plt.plot(f1_s[:, c], label=trainer.dataset.class_names[c])
    plt.xlabel("Step"); plt.ylabel("F1")
    plt.title(f"F1 evolution during {'Training' if training else 'Inference'} (MA{window})")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    column_names = ['IoU', 'Precision', 'Recall', 'F1']
    row_names = [trainer.dataset.class_names[c] for c in range(trainer.dataset.num_classes) if trainer.dataset.class_names[c] not in ignore_class]
    row_names.append('Mean')

    iou_arr = np.asarray(stats['iou_per_class'])
    prec_arr = np.asarray(stats['precision_per_class'])
    recall_arr = np.asarray(stats['recall_per_class'])
    f1_arr = np.asarray(stats['f1_per_class'])

    data = [
        [iou_arr[:, c].mean(), prec_arr[:, c].mean(), recall_arr[:, c].mean(), f1_arr[:, c].mean()]
    for c in range(trainer.dataset.num_classes) if trainer.dataset.class_names[c] not in ignore_class]

    means = np.array(data).mean(axis=0)
    data.append(list(means))

    df = pd.DataFrame(data, columns=column_names, index=row_names)
    display(df)

In [ ]:
tester = TreeProjectorTrainer(
    dataset_folder=DATASET_FOLDER,
    voxel_size=VOXEL_SIZE,
    train_pct=TRAIN_PCT,
    data_augmentation_coef=DATA_AUGMENTATION_COEF,
    epochs=EPOCHS,
    feat_keys=FEAT_KEYS,
    channels=CHANNELS,
    latent_dim=LATENT_DIM,
    instance_density=INSTANCE_DENSITY,
    centroid_thres=CENTROID_THRES,
    peak_radius=PEAK_RADIUS,
    min_score_for_center=MIN_SCORE_FOR_CENTROID,
    descriptor_dim=DESCRIPTOR_DIM,
    batch_size=BATCH_SIZE,
    training=TRAINING,
    semantic_loss_coef=SEMANTIC_LOSS_COEF,
    centroid_loss_coef=CENTROID_LOSS_COEF,
    instance_loss_coef=INSTANCE_LOSS_COEF,
    weights_file=WEIGHTS_FILE,
    checkpoint_file=CHECKPOINT_FILE
)

if TRAINING:
    tester.train()
    stats = tester.stats
    losses = tester.losses
    gen_charts(tester, losses, stats, True, ignore_class=IGNORE_CLASS)

pcd = o3d.geometry.PointCloud()
pcd2 = o3d.geometry.PointCloud()
for voxels, semantic_output, semantic_labels, centroid_score_output, centroid_score_labels, instance_output, instance_labels, centroid_voxels, centroid_confidence_output in tester.eval():
    batch_idx = voxels[:, 0]
    centroid_batch_idx = centroid_voxels[:, 0]
    voxels = voxels[:, 1:]
    centroid_voxels = centroid_voxels[:, 1:]

    for idx in np.unique(batch_idx):
        mask = batch_idx == idx
        cloud_voxels = voxels[mask]
        cloud_semantic_output = semantic_output[mask]
        cloud_semantic_labels = semantic_labels[mask]
        cloud_centroid_score_output = centroid_score_output[mask]
        cloud_centroid_score_labels = centroid_score_labels[mask]
        cloud_instance_output = instance_output[mask]
        cloud_instance_labels = instance_labels[mask]

        mask = centroid_batch_idx == idx
        cloud_centroid_voxels = centroid_voxels[mask]
        cloud_centroid_confidence_output = centroid_confidence_output[mask]

        pcd.points = o3d.utility.Vector3dVector(cloud_voxels)

        colors = tester.dataset.class_colormap[cloud_semantic_labels] / 255.0
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd])

        colors = tester.dataset.class_colormap[cloud_semantic_output] / 255.0
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd])

        cmap = plt.get_cmap('viridis')

        colors = cmap(cloud_centroid_score_labels[:, 0])[:, :3]
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd])

        colors = cmap(cloud_centroid_score_output[:, 0])[:, :3]
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd])

        spheres = []
        colors = tester.dataset.class_colormap[cloud_semantic_labels] / 255.0
        pcd.colors = o3d.utility.Vector3dVector(colors)
        for i in range(cloud_centroid_voxels.shape[0]):
            center = cloud_centroid_voxels[i]
            confidence = cloud_centroid_confidence_output[i][0]

            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.5)
            sphere.translate(center)
            color = cmap(confidence)[:3]
            sphere.paint_uniform_color(color)
            spheres.append(sphere)

        o3d.visualization.draw_geometries([pcd] + spheres)

        unique_ids = np.unique(cloud_instance_labels)
        rng = np.random.default_rng(0)
        palette = rng.random((len(unique_ids), 3))

        id2color = {uid: palette[i] for i, uid in enumerate(unique_ids)}
        colors = np.array([id2color[i] for i in cloud_instance_labels], dtype=np.float64)

        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd])

        unique_ids = np.unique(cloud_instance_output)
        rng = np.random.default_rng(0)
        palette = rng.random((len(unique_ids), 3))

        id2color = {uid: palette[i] for i, uid in enumerate(unique_ids)}
        colors = np.array([id2color[i] for i in cloud_instance_output], dtype=np.float64)

        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd])

gen_charts(tester, tester.losses, tester.stats, False, ignore_class=IGNORE_CLASS)


tf32 enabled: False
Parámetros totales: 12,794,062
Parámetros entrenables: 12,794,062

=== Starting epoch 1 ===
Training instance correlation with labels instead of predictions by now...



[Train]:   0%|          | 0/3936 [00:08<?, ?it/s]


RuntimeError: The size of tensor a (5536) must match the size of tensor b (792) at non-singleton dimension 0